In [2]:
import pandas as pd
import os
import numpy as np
from multiprocessing import cpu_count
from tqdm.auto import tqdm
import csv as csv
np.random.seed(42)
cpu_count()

8

In [2]:
TRAIN_FILES = [
    './data/UNSW-NB15_1.csv',
    './data/UNSW-NB15_2.csv',
    './data/UNSW-NB15_3.csv',
]

In [4]:
column_names = [
    'srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sloss',
    'dloss', 'service', 'Sload', 'Dload', 'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz', 'dmeansz',
    'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Stime', 'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack',
    'ackdat', 'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src',
    'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
    'attack_cat', 'Label'
]

In [4]:
df = pd.read_csv('./data/UNSW-NB15_1.csv', header=None, names=column_names, low_memory=False)
csv = df.drop(['srcip','dstip','attack_cat', 'Stime', 'Ltime'], axis=1)
x = df.drop(['Label'], axis=1)
y = df['Label']

In [9]:
def load_csv(name):
    df = pd.read_csv(name, header=None, names=column_names, low_memory=False)
    df = df.drop(['srcip','dstip','attack_cat', 'Stime', 'Ltime'], axis=1)
    return df

In [12]:
files_count = 0
output_dir = './csv_fold'
os.makedirs(output_dir, exist_ok=True)
header = [
    'sport', 'dsport', 'proto', 'state', 'dur', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sloss',
    'dloss', 'service', 'Sload', 'Dload', 'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz', 'dmeansz',
    'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack',
    'ackdat', 'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src',
    'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
    'Label'
]
output_file = os.path.join(output_dir, f"fold_{files_count}.csv")

for file in tqdm(TRAIN_FILES):
    df = load_csv(file)
    # write to csv, 100000 rows per file
    line_count = 0
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        for index, row in df.iterrows():
            writer.writerow(row)
            line_count += 1
            if line_count >= 100000:
                line_count = 0
                files_count += 1
                output_file = os.path.join(output_dir, f"fold_{files_count}.csv")
                f = open(output_file, 'w', newline='')
                writer = csv.writer(f)
                writer.writerow(header)

print(f"Total files: {files_count}")

  0%|          | 0/3 [00:00<?, ?it/s]

Total files: 21


In [5]:
test_csv = pd.read_csv('./data/UNSW-NB15_4.csv', header=None, names=column_names, low_memory=False)

In [6]:
test_csv = test_csv.drop(['srcip','dstip','attack_cat', 'Stime', 'Ltime'], axis=1)

In [7]:
test_csv.to_csv('./csv_fold/test.csv', index=False)